In [2]:
import os
from dotenv import load_dotenv
import openai
import requests
import json
import time
import logging
from datetime import datetime


In [3]:
# Load environment variables
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
# Initialize OpenAI client
client = openai.OpenAI()
model = "gpt-4o-mini"

In [5]:
# Create Contract Checking Assistant
instructions = """
You are a Contract Checking Assistant. Your task is to receive written contracts, check them against specified regulations, identify missing parts, suggest modifications, and provide recommendations for improvement.

When a contract is provided, follow these steps:

1. **Check for Compliance**:
    - Verify that the contract complies with Iran's regulations.
    - Check compliance with ICC and Incoterms regulations.
2. **Identify Missing Parts**:
    - Highlight any sections that are missing or incomplete.
3. **Suggest Modifications**:
    - Recommend changes to ensure the contract meets all legal and regulatory requirements.
4. **Provide Improvements**:
    - Offer suggestions on how to improve the contract for clarity, fairness, and comprehensiveness.
5. **Compare with Similar Contracts**:
    - Compare the contract with similar contracts to identify common practices and potential improvements.
6. **Identify Weaknesses and Challenges**:
    - Find weaknesses and challenging points in the contract and suggest modifications for these parts.

Your responses should be clear, concise, and professional. Always provide detailed explanations for your suggestions and ensure that your feedback is actionable.
"""


In [6]:
Contract_Checking_Assistant = client.beta.assistants.create(
    name="Contract_Check_Assistant_V3",
    instructions=instructions,
    model=model,
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"},
            ],
    metadata={"project": "Contract Review"}
)
print(Contract_Checking_Assistant)
print(Contract_Checking_Assistant.id)

Assistant(id='asst_qlnzqe9DupI93qn9MdFPhlZJ', created_at=1725890478, description=None, instructions="\nYou are a Contract Checking Assistant. Your task is to receive written contracts, check them against specified regulations, identify missing parts, suggest modifications, and provide recommendations for improvement.\n\nWhen a contract is provided, follow these steps:\n\n1. **Check for Compliance**:\n    - Verify that the contract complies with Iran's regulations.\n    - Check compliance with ICC and Incoterms regulations.\n2. **Identify Missing Parts**:\n    - Highlight any sections that are missing or incomplete.\n3. **Suggest Modifications**:\n    - Recommend changes to ensure the contract meets all legal and regulatory requirements.\n4. **Provide Improvements**:\n    - Offer suggestions on how to improve the contract for clarity, fairness, and comprehensiveness.\n5. **Compare with Similar Contracts**:\n    - Compare the contract with similar contracts to identify common practices a

In [7]:
# Create a vector store called "The Rules"
vector_store = client.beta.vector_stores.create(name="The Rules")

In [8]:
# Ready the files for upload to OpenAI
file_paths = ["./ICCtraderules.pdf", "./Irantraderules.pdf", "./insurancerules.pdf", "./similarcontracts.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

In [9]:
# Upload files to the vector store and poll the status
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

In [10]:
# Print the status and file counts of the batch
print(file_batch.status)
print(file_batch.file_counts)

failed
FileCounts(cancelled=0, completed=0, failed=4, in_progress=0, total=4)


In [12]:
# Update the assistant with the vector store
Contract_Checking_Assistant = client.beta.assistants.update(
    assistant_id=Contract_Checking_Assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)


In [37]:


# Create a sample thread with a non-empty message content
sample_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": 'Please check the contract against the specified regulations.'
        }
    ]
)

print("Sample Thread ID:")
print(sample_thread.id)


Sample Thread ID:
thread_At3XKKUUCbZemLhvkLpZcem8
